# VARIOS SEQUENCE TO SEQUENCE ARCHITECTURES

### Basic models

- machine translation: Input $\{x^{<1>}\}$, $\{x^{<2>}\}$ and output are $\{y^{<1>}\}$ ... $\{y^{<2>}\}$.  

Start by building an incput, `encoder network` (RNN (GRU or LSTM), that takes one word at a time). After that build a `decoder netword` that outputs a sequence $\{y^{<1>}\}$ ... $\{y^{<2>}\}$.

This model words given enough pairs of sentences of different words.  This is `encoder-decoder` network.  

A similar architecture words for image captioning. 
The netword takes image via CNN (pre-train net), an learn a set of feautres of an image. It returns a dense layer with a lot of units. This can be a _encoder netword_. This can then be fed into _RNN_, a _decoder network_ to generate a caption. This works reasonably well.  

These are basic seq-to-seq.  but it gives a randomly chosen output, not _the most likely output_.  

### Picking the most likely sequence

Machine trasnlation as a conditional language model.  

A language model returns a probability of a given output.  

- Conditional language model. In a classical language model the inputs $a^{<0>}$ are set as zero. Here we use the output of the _encoder NN_ to _condition_ the language model. Thus, machine translation becomes _encoder and decoder_ NNs. The _decoder_ NN is has an output similar to the _language model_ but the _encoder_ is used to condition it. 

The model gives probabilities of possible translation sentences.  It is important to _avoid sampling randomly_ from the distribution of outputs.  
Find a sentence, output, that maximises the conditional ptobability $x$.  This is done via `beam search`. 
The `gready search` _is not generally used_ (where word by word are picked based on their probabilities). Instead, _joint probabilities_ of _all words in the sentence_ is used. 

Also, _approximate search_ is used as the sample of search is too large.  

# Beam search

Choosing the best ouput from LLMs.  

Beam search picks first the first most likely word.  
Chose the probability of various outputs given an input. The Search outputs _several_ likely outputs. The number is given by $B$ variable.  

Next, for each of the _selected_ outputs, consider the probabilities for the next word (remember we feed the previous word into the next layer of the _decoder_, so for each previous output, the next one will change).  

Now we collect pairs of first-second words that are most likely. The selsection is done as 
$$
P(y^{<1>},y^{<2>}|x) = P(y^{<1>}|x)P(y^{<2>}|x,y^{<1>})
$$
The same is done for the third etc word. This allows to select the _most probable setnece_ rather than word by word, contrary to the _greedy search_ (Note, at $B=1$ it is a greedy search)

Beam search aims to maximize 

$$
\text{arg max} \,\Pi_{t=1}^{T_y} P(y^{<t>}|x,y^{<1>}...y^{<t-1>})
$$

### Refinments to beam search 
- Length normalization

More numerically stable algorithm can be obtained by considering $\sum\log(P(...))$ instead of product of probabilities.  

This function however, prefers short sentences, as it is simpler to optimize for them.  
A common approach is to add 
$$
\text{arg max} \frac{1}{T_y^{\alpha}} \,\sum_{t=1}^{T_y} \log P(y^{<t>}|x,y^{<1>}...y^{<t-1>})
$$

where the first time here accounts for the equation bias for short sentences. 

This a _heuristic approach_. 

IF $B$ is large -- it is slower, more memory needed, but it is more accurate. 
If $B$ is small, -- less accurate

In practice, $B\in(10-100)$

Beam search is _not exact search_ lke BFS or DFS (search algorithms)




### Error analysis in Beam Search

A model has two components: RNN and BeamSearch algorithm  
The error in translation can be attributed to either RNN or the beam search algorithm.  

Increasing the Beam Width may not increase the performance, as inclreasing the training dataset may not achieve this.  

RNN computes $P(y|x)$.  
Compare the result of RNN with human translation.  
Beam search choses $\hat{y}$. But $y^*$ gives higher $P(y|x)$, $P(y^*|x) > P(\hat{y}|x)$ Then, _beacm search fails_ as it fails to give the highest probability.  
If on the other hand $P(y^*|x) < P(\hat{y}|x)$, then RNN is the problem. 

Go through the dev.set and find the mistackes that algorithm made. 

Compare the probabiltiies for $*$ and $\hat{}$ and what gives more error, beam search or RNN.  
Then the error analysis is what fraction attributed to different algorithm.  

The beam search is worth checking only if it is responsible for most of the errors. 


### Bleu Score (Bilingual evaluation)

Evaluating the machine translation when there are many valid answers.

Bleu score is the score that assess how close they are to human level.  

__PRecision__: how close each word to the expcted. Not usefull. 
__Modified Precision__: a word has a maximum number of times to be there. So, the word gives a cridit $2/7$, where number of appearances/number of words in the sentence.  

`Bigrams` - pairs of words appearining. 

Aan algorithm can compare these pairs in mahine outputs